In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("ISTA-DASLab/Llama-2-7b-AQLM-2Bit-2x8-hf")
model = AutoModelForCausalLM.from_pretrained("ISTA-DASLab/Llama-2-7b-AQLM-2Bit-2x8-hf")

/home/coralie/canada/Latent-truth-detector/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [12]:
import os
import torch
device = 'cuda' if torch.cuda.is_available() else 'CPU'
print('Device: {} '.format(device))

Device: cuda 


In [6]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): QuantizedLinear()
          (k_proj): QuantizedLinear()
          (v_proj): QuantizedLinear()
          (o_proj): QuantizedLinear()
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): QuantizedLinear()
          (up_proj): QuantizedLinear()
          (down_proj): QuantizedLinear()
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)

In [7]:
activations = {}
def capture_output(module, input, output):
    activations['layer_13_output'] = output[0].detach()
    
hook = model.model.layers[13].register_forward_hook(capture_output)

In [8]:
input_text = "paris is in france"
input_token = tokenizer.encode_plus(input_text, return_tensors="pt")

input_ids = input_token["input_ids"].to(device)
mask = input_token["attention_mask"].to(device)

In [9]:
with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=mask)
    
layer_13_output = activations['layer_13_output']
hook.remove()

In [10]:
layer_13_output.shape , input_ids

(torch.Size([1, 7, 4096]),
 tensor([[   1,  610,  275,  338,  297, 2524,  346]], device='cuda:0'))

- dimension 0 : taille du batch
- dimension 1 : taille de la séquence
- dimension 2 : taille de la représentation

On aura donc des tailles variables de sortie en fonction de la taille de la séquence d'entrée. 

Comment faire pour l'utiliser pour la classification ?

- utiliser le token `[CLS]` (modéle basé sur bert) qui est le premier token de la séquence (espece de représentation de la séquence entière) nous on est sur sententpiece `<s>` pas le meme role mais peut etre qu'on peut l'utiliser de la meme facon
- average pooling : moyenne des représentations des tokens de la séquence
- max pooling : max des représentations des tokens de la séquence
- system d'attention : on va donner un poids à chaque token de la séquence et on va faire une somme pondérée des représentations des tokens

In [11]:
encoded_input = tokenizer("Exemple de texte", return_tensors="pt")
print("Tokens encodés:", encoded_input)

tokenizer.decode(encoded_input["input_ids"].tolist()[0])

Tokens encodés: {'input_ids': tensor([[    1,  1222,   331,   552,   316, 19696,   371]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


'<s> Exemple de texte'